In [1]:
import datetime as dt

import pandas as pd
import numpy as np
from lenskit.algorithms import item_knn, user_knn
from sklearn.model_selection import ParameterGrid

from recsys4daos.datasets import to_lenskit
from recsys4daos.model_selection import cvtt_open, explore_hparams
from recsys4daos.evaluation import test_with_hparams_lenskit

import paths

/home/daviddavo/recsys4daos/src/recsys4daos/model_selection.py:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Parameters

In [2]:
# Dataset config
ORG_NAME = 'Decentraland'
SPLITS_FREQ = 'W-THU'  # Split weekly
LAST_FOLDS = 20  # Use just last 10 splits
SPLITS_NORMALIZE = True

# Evaluation
K_RECOMMENDATIONS: list[int] = [1,3,5,10,15,100]
SPLITS_FREQ: str = 'W-THU' # split weekly
SPLITS_NORMALIZE = True # Wether or not to move everything to 00:00
LAST_FOLDS = 10 # Use just last 10 splits
LAST_FOLD_DATE_STR: str = None

# Search space config
WINDOW_SIZES = ['7d', '14d', '21d', '30d', '60d', '90d', '10YE']
ITEMKNN_Ks = [1,2,3,4,5,6,7,8,9,10,15]

OPTIM_METRIC = 'map@10'

In [3]:
# Parameters
EXECUTION_ID = "2024-09-04T10:00"
ORG_NAME = "dOrg"
SPLITS_FREQ = "2d"
LAST_FOLDS = 10
SPLITS_NORMALIZE = True
LAST_FOLD_DATE_STR = "2022-02-18"


# Load the dataset

In [4]:
dfp = paths.load_proposals(ORG_NAME)
dfv = paths.load_votes(ORG_NAME)

print(dfp.info())
print(dfv.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343 entries, 0 to 342
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 343 non-null    object        
 1   author             343 non-null    object        
 2   date               343 non-null    datetime64[us]
 3   start              343 non-null    datetime64[us]
 4   end                342 non-null    datetime64[us]
 5   platform_proposal  343 non-null    object        
dtypes: datetime64[us](3), object(3)
memory usage: 16.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1782 entries, 0 to 1781
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        1782 non-null   object        
 1   proposal  1782 non-null   object        
 2   voter     1782 non-null   object        
 3   date      1782 non-null   datetime64[us]
dtypes: d

In [5]:
df = to_lenskit(dfv)
df

,user,item,timestamp,rating
0,0x7f2a1ef39ea0d470b5669ce105a13c5b3bd8cd31,40babd7f-5215-581f-816b-f56d21d3f910,2020-03-06 16:18:05,1
1,0x36dbcbb5c90e9b00ab658353696610cc5d7b7002,40babd7f-5215-581f-816b-f56d21d3f910,2020-03-06 17:33:19,1
2,0xb1b7586656116d546033e3baff69bfcd6592225e,40babd7f-5215-581f-816b-f56d21d3f910,2020-03-06 19:59:20,1
3,0xb1b7586656116d546033e3baff69bfcd6592225e,616fe9b9-c5f2-5571-904c-fb0c09f04e19,2020-03-09 02:15:26,1
4,0x7f2a1ef39ea0d470b5669ce105a13c5b3bd8cd31,616fe9b9-c5f2-5571-904c-fb0c09f04e19,2020-03-12 00:02:44,1
...,...,...,...,...
1777,0x6fe74660d9ccad62f5ce15657e95453f1c54b6fb,ccbd62f1-63d3-5db8-95fd-7854f034bab2,2023-06-29 14:42:01,1
1778,0x04181a9feec83a9692f2126333660e7a0cf13a73,ccbd62f1-63d3-5db8-95fd-7854f034bab2,2023-06-30 15:05:47,1
1779,0xc5d5851a81afc92d49266f73824c2701730b8a6d,ccbd62f1-63d3-5db8-95fd-7854f034bab2,2023-06-30 16:55:15,1
1780,0xd9d41335fee80ede3ba22f33100740c2858f761a,ccbd62f1-63d3-5db8-95fd-7854f034bab2,2023-07-01 14:27:48,1


## Split in folds

In [6]:
all_folds = { f.end:f for f in cvtt_open(
    df, SPLITS_FREQ, dfp.reset_index(), remove_not_in_train_col='item', col_item='item', last_fold=LAST_FOLD_DATE_STR,
)}
last_folds_idx = list(all_folds.keys())[-LAST_FOLDS:]
last_folds_idx

[Timestamp('2022-01-31 00:00:00'),
 Timestamp('2022-02-02 00:00:00'),
 Timestamp('2022-02-04 00:00:00'),
 Timestamp('2022-02-06 00:00:00'),
 Timestamp('2022-02-08 00:00:00'),
 Timestamp('2022-02-10 00:00:00'),
 Timestamp('2022-02-12 00:00:00'),
 Timestamp('2022-02-14 00:00:00'),
 Timestamp('2022-02-16 00:00:00'),
 Timestamp('2022-02-18 00:00:00')]

# Item-based KNN

In [7]:
def testHParamsItemKNN(fold, k: int, window_size=None):
    # Create algorithm
    algo = item_knn.ItemItem(
        feedback='implicit', # VERY IMPORTANT
        min_sim=0,
        # min_nbrs=0,
        nnbrs=k,  # the maximum number of neighbors for scoring each item (None for unlimited)
    )
    return test_with_hparams_lenskit(algo, all_folds[fold], K_RECOMMENDATIONS, window_size)

pd.Series(testHParamsItemKNN(last_folds_idx[-1],5,'14d'))

/home/daviddavo/recsys4daos/.direnv/python-3.11/lib/python3.11/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
Numba is using threading layer omp - consider TBB


found 1 potential runtime problems - see https://boi.st/lkpy-perf


/home/daviddavo/recsys4daos/.direnv/python-3.11/lib/python3.11/site-packages/lenskit/algorithms/item_knn.py:119: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  b = blocks[bi]


fold_t             2022-02-18 00:00:00
time_train                    5.472681
time_rec                      0.004714
open_proposals                       2
min_recs                             1
avg_recs                           1.5
precision@1                        0.5
precision@3                   0.333333
precision@5                        0.2
precision@10                       0.1
precision@15                  0.066667
precision@100                     0.01
ndcg@1                             0.5
ndcg@3                        0.815465
ndcg@5                        0.815465
ndcg@10                       0.815465
ndcg@15                       0.815465
ndcg@100                      0.815465
map@1                              0.5
map@3                             0.75
map@5                             0.75
map@10                            0.75
map@15                            0.75
map@100                           0.75
recall@1                           0.5
recall@3                 

## Exploring hparams

In [8]:
results = explore_hparams(
    testHParamsItemKNN, 
    ParameterGrid({
    'fold': last_folds_idx,
    'k': ITEMKNN_Ks, # ,20,25,30,35,40,45,50,60,70,80,90,100],
    'window_size': WINDOW_SIZES,
    }),
    paths.hparams_progress('itemknn', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE),
)
mdfi = pd.DataFrame(results)
mdfi

Restored checkpoint from ../.cache/dOrg/hparams-itemknn_2d_normalize.pkl with 2310 results


  0%|          | 0/770 [00:00<?, ?it/s]

,fold,k,window_size,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
0,2022-01-31,1,7d,2022-01-31,0.005686,0.011517,4,1,1.900000,0.500000,...,1.0,1.0,1.0,1.0,0.857143,0.857143,0.857143,0.857143,0.857143,0.123442
1,2022-01-31,1,14d,2022-01-31,0.005660,0.013318,4,1,2.250000,0.416667,...,1.0,1.0,1.0,1.0,0.857143,0.857143,0.857143,0.857143,0.857143,0.123666
2,2022-01-31,1,21d,2022-01-31,0.015093,0.013953,4,1,2.250000,0.416667,...,1.0,1.0,1.0,1.0,0.857143,0.857143,0.857143,0.857143,0.857143,0.123570
3,2022-01-31,1,30d,2022-01-31,0.011567,0.013978,4,1,2.250000,0.416667,...,1.0,1.0,1.0,1.0,0.857143,0.857143,0.857143,0.857143,0.857143,0.123570
4,2022-01-31,1,60d,2022-01-31,0.005752,0.015595,4,1,2.384615,0.384615,...,1.0,1.0,1.0,1.0,0.857143,0.857143,0.857143,0.857143,0.857143,0.123923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,2022-02-18,15,21d,2022-02-18,0.011527,0.003443,2,1,1.500000,0.500000,...,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.128768
766,2022-02-18,15,30d,2022-02-18,0.010388,0.003476,2,1,1.500000,0.500000,...,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.128258
767,2022-02-18,15,60d,2022-02-18,0.005989,0.003788,2,1,1.500000,0.500000,...,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.129229
768,2022-02-18,15,90d,2022-02-18,0.001635,0.003824,2,1,1.500000,0.500000,...,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.131849


### Best overall hparams

In [9]:
display_columns = ['time_train', 'avg_recs'] + [ c for c in mdfi.columns if c.endswith('@5') or c.endswith('@10') or c.endswith('@100') ]
overall_hparams = mdfi[mdfi['fold'] > last_folds_idx[0]].groupby(['window_size', 'k']).mean().sort_values(OPTIM_METRIC, ascending=False)
overall_hparams[display_columns]

time_train  avg_recs  precision@5  precision@10  \
window_size k                                                     
7d          1     0.005029  1.572222     0.222222      0.111111   
            5     0.003212  1.572222     0.222222      0.111111   
            10    0.008116  1.572222     0.222222      0.111111   
            3     0.005339  1.572222     0.222222      0.111111   
            9     0.004227  1.572222     0.222222      0.111111   
...                    ...       ...          ...           ...   
90d         6     0.002304  1.755026     0.218201      0.109101   
60d         5     0.007803  1.755026     0.218201      0.109101   
30d         1     0.003702  1.740741     0.219259      0.109630   
90d         1     0.003258  1.755026     0.218201      0.109101   
60d         1     0.003591  1.755026     0.218201      0.109101   

                precision@100    ndcg@5   ndcg@10  ndcg@100     map@5  \
window_size k                                                           
7d          1        0.011111  0.618607  0.618607  0.618607  0.820370   
            5        0.011111  0.618607  0.618607  0.618607  0.820370   
            10       0.011111  0.618607  0.618607  0.618607  0.820370   
            3        0.011111  0.618607  0.618607  0.618607  0.820370   
            9        0.011111  0.618607  0.618607  0.618607  0.820370   
...                       ...       ...       ...       ...       ...   
90d         6        0.010910  0.772101  0.772101  0.772101  0.723251   
60d         5        0.010910  0.772101  0.772101  0.772101  0.723251   
30d         1        0.010963  0.729973  0.729973  0.729973  0.720782   
90d         1        0.010910  0.768521  0.768521  0.768521  0.718401   
60d         1        0.010910  0.768521  0.768521  0.768521  0.718401   

                  map@10   map@100  recall@5  recall@10  recall@100  \
window_size k                                                         
7d          1   0.820370  0.820370       1.0        1.0         1.0   
            5   0.820370  0.820370       1.0        1.0         1.0   
            10  0.820370  0.820370       1.0        1.0         1.0   
            3   0.820370  0.820370       1.0        1.0         1.0   
            9   0.820370  0.820370       1.0        1.0         1.0   
...                  ...       ...       ...        ...         ...   
90d         6   0.723251  0.723251       1.0        1.0         1.0   
60d         5   0.723251  0.723251       1.0        1.0         1.0   
30d         1   0.720782  0.720782       1.0        1.0         1.0   
90d         1   0.718401  0.718401       1.0        1.0         1.0   
60d         1   0.718401  0.718401       1.0        1.0         1.0   

                r-precision@5  r-precision@10  r-precision@100  
window_size k                                                   
7d          1         1.00000         1.00000          1.00000  
            5         1.00000         1.00000          1.00000  
            10        1.00000         1.00000          1.00000  
            3         1.00000         1.00000          1.00000  
            9         1.00000         1.00000          1.00000  
...                       ...             ...              ...  
90d         6         0.96875         0.96875          0.96875  
60d         5         0.96875         0.96875          0.96875  
30d         1         0.96875         0.96875          0.96875  
90d         1         0.96875         0.96875          0.96875  
60d         1         0.96875         0.96875          0.96875  

[77 rows x 17 columns]

Now let's see the behaviour in each fold

In [10]:
best_avg_hparams = mdfi.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])
paths.save_model_results(best_avg_hparams, 'itemknn-best-avg', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_avg_hparams

Saved dataframe into /home/daviddavo/recsys4daos/data/output/dOrg/models/itemknn-best-avg_2d_normalize.parquet


/tmp/ipykernel_3821226/3951593685.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  best_avg_hparams = mdfi.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])


,,,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
fold,window_size,k,,,,,,,,,,,,,,,,,,,,,
2022-01-31,7d,1,2022-01-31,0.005686,0.011517,4,1,1.900000,0.500000,0.400000,0.24,0.12,...,1.0,1.0,1.0,1.000000,0.857143,0.857143,0.857143,0.857143,0.857143,0.123442
2022-02-02,7d,1,2022-02-02,0.001587,0.146802,3,2,2.000000,0.000000,0.333333,0.20,0.10,...,1.0,1.0,1.0,0.000000,NaN,NaN,NaN,NaN,NaN,0.126788
2022-02-04,7d,1,2022-02-04,0.008564,0.149247,2,1,1.400000,0.600000,0.333333,0.20,0.10,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.136197
2022-02-06,7d,1,2022-02-06,0.010373,0.005410,2,1,1.750000,1.000000,0.500000,0.30,0.15,...,1.0,1.0,1.0,0.750000,1.000000,1.000000,1.000000,1.000000,1.000000,0.131779
2022-02-08,7d,1,2022-02-08,0.001034,0.007471,3,1,2.000000,0.666667,0.500000,0.30,0.15,...,1.0,1.0,1.0,0.708333,1.000000,1.000000,1.000000,1.000000,1.000000,0.133314
2022-02-10,7d,1,2022-02-10,0.001042,0.004236,3,1,1.666667,0.333333,0.333333,0.20,0.10,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.130028
2022-02-12,7d,1,2022-02-12,0.005721,0.004171,2,1,1.333333,0.666667,0.333333,0.20,0.10,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.132461
2022-02-14,7d,1,2022-02-14,0.009655,0.004128,1,1,1.000000,1.000000,0.333333,0.20,0.10,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.130048
2022-02-16,7d,1,2022-02-16,0.005744,0.003022,2,1,1.500000,0.500000,0.333333,0.20,0.10,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.135282


### Best hparams by fold

This are the parameters used to check which model is the best (validation)

In [11]:
print("Best hyperparams by fold", OPTIM_METRIC)
best_hparams = mdfi.sort_values(OPTIM_METRIC,ascending=False).drop_duplicates(['fold'], keep='first').sort_values('fold').set_index(['fold', 'k', 'window_size'])
paths.save_model_results(best_hparams, 'itemknn-best-val', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_hparams[display_columns]

Best hyperparams by fold map@10
Saved dataframe into /home/daviddavo/recsys4daos/data/output/dOrg/models/itemknn-best-val_2d_normalize.parquet


,,,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
fold,k,window_size,,,,,,,,,,,,,,,,,
2022-01-31,1,7d,0.005686,1.900000,0.24,0.12,0.012,0.509295,0.509295,0.509295,0.750000,0.750000,0.750000,1.0,1.0,1.0,0.857143,0.857143,0.857143
2022-02-02,6,10YE,0.016565,2.333333,0.20,0.10,0.010,0.753953,0.753953,0.753953,0.666667,0.666667,0.666667,1.0,1.0,1.0,1.000000,1.000000,1.000000
2022-02-04,15,7d,0.008513,1.400000,0.20,0.10,0.010,0.608837,0.608837,0.608837,0.800000,0.800000,0.800000,1.0,1.0,1.0,1.000000,1.000000,1.000000
2022-02-06,1,7d,0.010373,1.750000,0.30,0.15,0.015,0.571429,0.571429,0.571429,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.000000
2022-02-08,7,7d,0.009813,2.000000,0.30,0.15,0.015,0.584651,0.584651,0.584651,0.833333,0.833333,0.833333,1.0,1.0,1.0,1.000000,1.000000,1.000000
2022-02-10,4,7d,0.000959,1.666667,0.20,0.10,0.010,0.452372,0.452372,0.452372,0.666667,0.666667,0.666667,1.0,1.0,1.0,1.000000,1.000000,1.000000
2022-02-12,9,14d,0.001014,1.333333,0.20,0.10,0.010,0.657732,0.657732,0.657732,0.833333,0.833333,0.833333,1.0,1.0,1.0,1.000000,1.000000,1.000000
2022-02-14,2,7d,0.005631,1.000000,0.20,0.10,0.010,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.000000
2022-02-16,3,60d,0.005882,1.833333,0.20,0.10,0.010,0.815465,0.815465,0.815465,0.750000,0.750000,0.750000,1.0,1.0,1.0,1.000000,1.000000,1.000000


In [12]:
best_hparams.tail(len(best_hparams)-1).describe()

,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
count,9,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.0,9.0,9.0,9.000000,9.0,9.0,9.0,9.0,9.0,9.000000
mean,2022-02-10 00:00:00,0.006699,0.005659,2.222222,1.111111,1.646296,0.677778,0.370370,0.222222,0.111111,...,1.0,1.0,1.0,0.939815,1.0,1.0,1.0,1.0,1.0,0.130392
min,2022-02-02 00:00:00,0.000959,0.003026,1.000000,1.000000,1.000000,0.333333,0.333333,0.200000,0.100000,...,1.0,1.0,1.0,0.708333,1.0,1.0,1.0,1.0,1.0,0.127861
25%,2022-02-06 00:00:00,0.001537,0.004174,2.000000,1.000000,1.400000,0.500000,0.333333,0.200000,0.100000,...,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,0.129286
50%,2022-02-10 00:00:00,0.005882,0.005410,2.000000,1.000000,1.666667,0.666667,0.333333,0.200000,0.100000,...,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,0.130381
75%,2022-02-14 00:00:00,0.009813,0.006812,3.000000,1.000000,1.833333,1.000000,0.333333,0.200000,0.100000,...,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,0.131054
max,2022-02-18 00:00:00,0.016565,0.008943,3.000000,2.000000,2.333333,1.000000,0.500000,0.300000,0.150000,...,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,0.135060
std,NaN,0.005215,0.001924,0.666667,0.333333,0.393857,0.270801,0.073493,0.044096,0.022048,...,0.0,0.0,0.0,0.119880,0.0,0.0,0.0,0.0,0.0,0.002173


### Results of using best hparams in next fold

Kind of like the cvtt from [the LightGCN notebook](./11_microsoft_tuning.ipynb).

In [13]:
next_hparams_idx = best_hparams.index.to_frame(False)
next_hparams_idx['fold'] = next_hparams_idx['fold'].shift(-1)
next_hparams_idx = next_hparams_idx.dropna()
assert len(next_hparams_idx) == len(best_hparams)-1
next_hparams = mdfi.set_index(['fold', 'k', 'window_size']).loc[pd.MultiIndex.from_frame(next_hparams_idx)]
paths.save_model_results(next_hparams, 'itemknn-best-test', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
next_hparams[display_columns]

Saved dataframe into /home/daviddavo/recsys4daos/data/output/dOrg/models/itemknn-best-test_2d_normalize.parquet


,,,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
fold,k,window_size,,,,,,,,,,,,,,,,,
2022-02-02,1,7d,0.001587,2.000000,0.2,0.10,0.010,0.420620,0.420620,0.420620,0.500000,0.500000,0.500000,1.0,1.0,1.0,NaN,NaN,NaN
2022-02-04,6,10YE,0.007073,1.571429,0.2,0.10,0.010,0.789103,0.789103,0.789103,0.714286,0.714286,0.714286,1.0,1.0,1.0,1.0,1.0,1.0
2022-02-06,15,7d,0.001037,1.750000,0.3,0.15,0.015,0.571429,0.571429,0.571429,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0
2022-02-08,1,7d,0.001034,2.000000,0.3,0.15,0.015,0.584651,0.584651,0.584651,0.833333,0.833333,0.833333,1.0,1.0,1.0,1.0,1.0,1.0
2022-02-10,7,7d,0.014162,1.666667,0.2,0.10,0.010,0.452372,0.452372,0.452372,0.666667,0.666667,0.666667,1.0,1.0,1.0,1.0,1.0,1.0
2022-02-12,4,7d,0.001022,1.333333,0.2,0.10,0.010,0.657732,0.657732,0.657732,0.833333,0.833333,0.833333,1.0,1.0,1.0,1.0,1.0,1.0
2022-02-14,9,14d,0.001868,1.000000,0.2,0.10,0.010,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0
2022-02-16,2,7d,0.010401,1.500000,0.2,0.10,0.010,0.271822,0.271822,0.271822,0.750000,0.750000,0.750000,1.0,1.0,1.0,1.0,1.0,1.0
2022-02-18,3,60d,0.015029,1.500000,0.2,0.10,0.010,0.815465,0.815465,0.815465,0.750000,0.750000,0.750000,1.0,1.0,1.0,1.0,1.0,1.0


In [14]:
next_hparams.describe()

,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
count,9,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.0,9.0,9.0,9.000000,8.0,8.0,8.0,8.0,8.0,9.000000
mean,2022-02-10 00:00:00,0.005913,0.021250,2.222222,1.111111,1.591270,0.566138,0.370370,0.222222,0.111111,...,1.0,1.0,1.0,0.828704,1.0,1.0,1.0,1.0,1.0,0.130831
min,2022-02-02 00:00:00,0.001022,0.003029,1.000000,1.000000,1.000000,0.000000,0.333333,0.200000,0.100000,...,1.0,1.0,1.0,0.000000,1.0,1.0,1.0,1.0,1.0,0.126788
25%,2022-02-06 00:00:00,0.001037,0.004114,2.000000,1.000000,1.500000,0.428571,0.333333,0.200000,0.100000,...,1.0,1.0,1.0,0.750000,1.0,1.0,1.0,1.0,1.0,0.128603
50%,2022-02-10 00:00:00,0.001868,0.004388,2.000000,1.000000,1.571429,0.500000,0.333333,0.200000,0.100000,...,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,0.131314
75%,2022-02-14 00:00:00,0.010401,0.007471,3.000000,1.000000,1.750000,0.666667,0.333333,0.200000,0.100000,...,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,0.132982
max,2022-02-18 00:00:00,0.015029,0.146802,3.000000,2.000000,2.000000,1.000000,0.500000,0.300000,0.150000,...,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0,1.0,0.135354
std,NaN,0.005908,0.047164,0.666667,0.333333,0.315869,0.316043,0.073493,0.044096,0.022048,...,0.0,0.0,0.0,0.332319,0.0,0.0,0.0,0.0,0.0,0.002773


# User-based KNN

In [15]:
def testHParamsUserKNN(fold, k: int, window_size=None):
    # Create algorithm
    algo = user_knn.UserUser(
        feedback='implicit', # VERY IMPORTANT
        min_sim=0,
        # min_nbrs=0,
        nnbrs=k,  # the maximum number of neighbors for scoring each item (None for unlimited)
    )
    return test_with_hparams_lenskit(algo, all_folds[fold], K_RECOMMENDATIONS, window_size)

pd.Series(testHParamsUserKNN(last_folds_idx[-1],5,'14d'))

fold_t             2022-02-18 00:00:00
time_train                    0.590625
time_rec                      0.831286
open_proposals                       2
min_recs                             1
avg_recs                           1.5
precision@1                        0.5
precision@3                   0.333333
precision@5                        0.2
precision@10                       0.1
precision@15                  0.066667
precision@100                     0.01
ndcg@1                             0.5
ndcg@3                        0.815465
ndcg@5                        0.815465
ndcg@10                       0.815465
ndcg@15                       0.815465
ndcg@100                      0.815465
map@1                              0.5
map@3                             0.75
map@5                             0.75
map@10                            0.75
map@15                            0.75
map@100                           0.75
recall@1                           0.5
recall@3                 

## Exploring hparams

In [16]:
results = explore_hparams(
    testHParamsUserKNN, 
    ParameterGrid({
        'fold': last_folds_idx,
        'k': ITEMKNN_Ks, # ,20,25,30,35,40,45,50,60,70,80,90,100],
        'window_size': WINDOW_SIZES,
    }),
    paths.hparams_progress('userknn', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE),
)
mdfu = pd.DataFrame(results)
mdfu

Restored checkpoint from ../.cache/dOrg/hparams-userknn_2d_normalize.pkl with 2310 results


  0%|          | 0/770 [00:00<?, ?it/s]

,fold,k,window_size,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
0,2022-01-31,1,7d,2022-01-31,0.000305,0.007246,4,1,1.900000,0.900000,...,1.0,1.0,1.0,0.888889,0.944444,0.944444,0.944444,0.944444,0.944444,0.123260
1,2022-01-31,1,14d,2022-01-31,0.000294,0.008562,4,1,2.250000,0.583333,...,1.0,1.0,1.0,0.928571,0.937500,0.937500,0.937500,0.937500,0.937500,0.123539
2,2022-01-31,1,21d,2022-01-31,0.000353,0.008611,4,1,2.250000,0.583333,...,1.0,1.0,1.0,0.928571,0.937500,0.937500,0.937500,0.937500,0.937500,0.123839
3,2022-01-31,1,30d,2022-01-31,0.000396,0.008613,4,1,2.250000,0.583333,...,1.0,1.0,1.0,0.928571,0.875000,0.875000,0.875000,0.875000,0.875000,0.125253
4,2022-01-31,1,60d,2022-01-31,0.000424,0.009451,4,1,2.384615,0.692308,...,1.0,1.0,1.0,0.888889,0.900000,0.900000,0.900000,0.900000,0.900000,0.123801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,2022-02-18,15,21d,2022-02-18,0.000423,0.002265,2,1,1.500000,0.500000,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.129396
766,2022-02-18,15,30d,2022-02-18,0.000430,0.002329,2,1,1.500000,0.500000,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.129304
767,2022-02-18,15,60d,2022-02-18,0.000481,0.002364,2,1,1.500000,0.500000,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.129581
768,2022-02-18,15,90d,2022-02-18,0.000507,0.002419,2,1,1.500000,0.500000,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.129561


### Best overall hparams

In [17]:
display_columns = ['time_train', 'avg_recs'] + [ c for c in mdfu.columns if c.endswith('@5') or c.endswith('@10') or c.endswith('@100') ]
overall_hparams = mdfu[mdfu['fold'] > last_folds_idx[0]].groupby(['window_size', 'k']).mean().sort_values(OPTIM_METRIC, ascending=False)
overall_hparams[display_columns]

time_train  avg_recs  precision@5  precision@10  \
window_size k                                                     
7d          1     0.000345  1.572222     0.222222      0.111111   
            2     0.000345  1.572222     0.222222      0.111111   
21d         1     0.000387  1.727778     0.221667      0.110833   
30d         1     0.000420  1.740741     0.219259      0.109630   
7d          15    0.000328  1.572222     0.222222      0.111111   
...                    ...       ...          ...           ...   
60d         8     0.000465  1.755026     0.218201      0.109101   
            9     0.000457  1.755026     0.218201      0.109101   
            10    0.000466  1.755026     0.218201      0.109101   
            15    0.000462  1.755026     0.218201      0.109101   
90d         15    0.000466  1.755026     0.218201      0.109101   

                precision@100    ndcg@5   ndcg@10  ndcg@100     map@5  \
window_size k                                                           
7d          1        0.011111  0.632276  0.632276  0.632276  0.848148   
            2        0.011111  0.632276  0.632276  0.632276  0.848148   
21d         1        0.011083  0.776248  0.776248  0.776248  0.831944   
30d         1        0.010963  0.802036  0.802036  0.802036  0.824691   
7d          15       0.011111  0.612749  0.612749  0.612749  0.806481   
...                       ...       ...       ...       ...       ...   
60d         8        0.010910  0.782516  0.782516  0.782516  0.737360   
            9        0.010910  0.782516  0.782516  0.782516  0.737360   
            10       0.010910  0.782516  0.782516  0.782516  0.737360   
            15       0.010910  0.782516  0.782516  0.782516  0.737360   
90d         15       0.010910  0.782516  0.782516  0.782516  0.737360   

                  map@10   map@100  recall@5  recall@10  recall@100  \
window_size k                                                         
7d          1   0.848148  0.848148       1.0        1.0         1.0   
            2   0.848148  0.848148       1.0        1.0         1.0   
21d         1   0.831944  0.831944       1.0        1.0         1.0   
30d         1   0.824691  0.824691       1.0        1.0         1.0   
7d          15  0.806481  0.806481       1.0        1.0         1.0   
...                  ...       ...       ...        ...         ...   
60d         8   0.737360  0.737360       1.0        1.0         1.0   
            9   0.737360  0.737360       1.0        1.0         1.0   
            10  0.737360  0.737360       1.0        1.0         1.0   
            15  0.737360  0.737360       1.0        1.0         1.0   
90d         15  0.737360  0.737360       1.0        1.0         1.0   

                r-precision@5  r-precision@10  r-precision@100  
window_size k                                                   
7d          1        1.000000        1.000000         1.000000  
            2        1.000000        1.000000         1.000000  
21d         1        0.988889        0.988889         0.988889  
30d         1        0.968750        0.968750         0.968750  
7d          15       1.000000        1.000000         1.000000  
...                       ...             ...              ...  
60d         8        0.968750        0.968750         0.968750  
            9        0.968750        0.968750         0.968750  
            10       0.968750        0.968750         0.968750  
            15       0.968750        0.968750         0.968750  
90d         15       0.968750        0.968750         0.968750  

[77 rows x 17 columns]

In [18]:
best_avg_hparams = mdfu.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])
paths.save_model_results(best_avg_hparams, 'userknn-best-avg', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_avg_hparams

Saved dataframe into /home/daviddavo/recsys4daos/data/output/dOrg/models/userknn-best-avg_2d_normalize.parquet


/tmp/ipykernel_3821226/3337470013.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  best_avg_hparams = mdfu.set_index(['window_size', 'k']).loc[overall_hparams.iloc[0].name].reset_index().set_index(['fold', 'window_size', 'k'])


,,,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
fold,window_size,k,,,,,,,,,,,,,,,,,,,,,
2022-01-31,7d,1,2022-01-31,0.000305,0.007246,4,1,1.900000,0.900000,0.400000,0.24,0.12,...,1.0,1.0,1.0,0.888889,0.944444,0.944444,0.944444,0.944444,0.944444,0.123260
2022-02-02,7d,1,2022-02-02,0.000487,0.002602,3,2,2.000000,0.500000,0.333333,0.20,0.10,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.134514
2022-02-04,7d,1,2022-02-04,0.000465,0.904539,2,1,1.400000,0.600000,0.333333,0.20,0.10,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.136500
2022-02-06,7d,1,2022-02-06,0.000305,0.003576,2,1,1.750000,1.000000,0.500000,0.30,0.15,...,1.0,1.0,1.0,0.750000,1.000000,1.000000,1.000000,1.000000,1.000000,0.132032
2022-02-08,7d,1,2022-02-08,0.000282,0.004983,3,1,2.000000,0.666667,0.500000,0.30,0.15,...,1.0,1.0,1.0,0.708333,1.000000,1.000000,1.000000,1.000000,1.000000,0.135858
2022-02-10,7d,1,2022-02-10,0.000321,0.002852,3,1,1.666667,0.333333,0.333333,0.20,0.10,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.132705
2022-02-12,7d,1,2022-02-12,0.000340,0.002828,2,1,1.333333,0.666667,0.333333,0.20,0.10,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.133529
2022-02-14,7d,1,2022-02-14,0.000309,0.002808,1,1,1.000000,1.000000,0.333333,0.20,0.10,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.129264
2022-02-16,7d,1,2022-02-16,0.000304,0.002210,2,1,1.500000,0.500000,0.333333,0.20,0.10,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.132363


### Best hparams by fold

In [19]:
print("Best hyperparams by fold", OPTIM_METRIC)
best_hparams = mdfu.sort_values(OPTIM_METRIC,ascending=False).drop_duplicates(['fold'], keep='first').sort_values('fold').set_index(['fold', 'k', 'window_size'])
paths.save_model_results(best_hparams, 'userknn-best-val', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
best_hparams[display_columns]

Best hyperparams by fold map@10
Saved dataframe into /home/daviddavo/recsys4daos/data/output/dOrg/models/userknn-best-val_2d_normalize.parquet


,,,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
fold,k,window_size,,,,,,,,,,,,,,,,,
2022-01-31,1,7d,0.000305,1.900000,0.240000,0.120000,0.012000,0.596916,0.596916,0.596916,0.933333,0.933333,0.933333,1.0,1.0,1.0,0.944444,0.944444,0.944444
2022-02-02,1,14d,0.000373,2.000000,0.200000,0.100000,0.010000,0.543643,0.543643,0.543643,0.750000,0.750000,0.750000,1.0,1.0,1.0,1.000000,1.000000,1.000000
2022-02-04,1,7d,0.000465,1.400000,0.200000,0.100000,0.010000,0.608837,0.608837,0.608837,0.800000,0.800000,0.800000,1.0,1.0,1.0,1.000000,1.000000,1.000000
2022-02-06,2,30d,0.000347,1.833333,0.266667,0.133333,0.013333,0.857143,0.857143,0.857143,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.000000
2022-02-08,1,30d,0.000580,2.333333,0.266667,0.133333,0.013333,0.917984,0.917984,0.917984,0.888889,0.888889,0.888889,1.0,1.0,1.0,1.000000,1.000000,1.000000
2022-02-10,2,10YE,0.000673,2.200000,0.240000,0.120000,0.012000,0.791057,0.791057,0.791057,0.716667,0.716667,0.716667,1.0,1.0,1.0,0.833333,0.833333,0.833333
2022-02-12,2,10YE,0.000692,1.500000,0.200000,0.100000,0.010000,0.907732,0.907732,0.907732,0.875000,0.875000,0.875000,1.0,1.0,1.0,1.000000,1.000000,1.000000
2022-02-14,5,90d,0.000533,1.000000,0.200000,0.100000,0.010000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.000000
2022-02-16,1,21d,0.000411,1.800000,0.200000,0.100000,0.010000,0.771822,0.771822,0.771822,0.900000,0.900000,0.900000,1.0,1.0,1.0,1.000000,1.000000,1.000000


In [20]:
best_hparams.tail(len(best_hparams)-1).describe()[display_columns]

,time_train,avg_recs,precision@5,precision@10,precision@100,ndcg@5,ndcg@10,ndcg@100,map@5,map@10,map@100,recall@5,recall@10,recall@100,r-precision@5,r-precision@10,r-precision@100
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.0,9.0,9.0,9.000000,9.000000,9.000000
mean,0.000495,1.729630,0.219259,0.109630,0.010963,0.822024,0.822024,0.822024,0.881173,0.881173,0.881173,1.0,1.0,1.0,0.981481,0.981481,0.981481
min,0.000347,1.000000,0.200000,0.100000,0.010000,0.543643,0.543643,0.543643,0.716667,0.716667,0.716667,1.0,1.0,1.0,0.833333,0.833333,0.833333
25%,0.000379,1.500000,0.200000,0.100000,0.010000,0.771822,0.771822,0.771822,0.800000,0.800000,0.800000,1.0,1.0,1.0,1.000000,1.000000,1.000000
50%,0.000465,1.800000,0.200000,0.100000,0.010000,0.857143,0.857143,0.857143,0.888889,0.888889,0.888889,1.0,1.0,1.0,1.000000,1.000000,1.000000
75%,0.000580,2.000000,0.240000,0.120000,0.012000,0.917984,0.917984,0.917984,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.000000
max,0.000692,2.333333,0.266667,0.133333,0.013333,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.000000,1.000000,1.000000
std,0.000131,0.421454,0.029897,0.014948,0.001495,0.161075,0.161075,0.161075,0.108207,0.108207,0.108207,0.0,0.0,0.0,0.055556,0.055556,0.055556


### Results of using best hparams in next fold

Kind of like the cvtt from [the LightGCN notebook](./11_microsoft_tuning.ipynb)

In [21]:
next_hparams_idx = best_hparams.index.to_frame(False)
next_hparams_idx['fold'] = next_hparams_idx['fold'].shift(-1)
next_hparams_idx = next_hparams_idx.dropna()
assert len(next_hparams_idx) == len(best_hparams)-1
next_hparams = mdfu.set_index(['fold', 'k', 'window_size']).loc[pd.MultiIndex.from_frame(next_hparams_idx)]
paths.save_model_results(next_hparams, 'userknn-best-test', ORG_NAME, SPLITS_FREQ, SPLITS_NORMALIZE, K_RECOMMENDATIONS)
next_hparams

Saved dataframe into /home/daviddavo/recsys4daos/data/output/dOrg/models/userknn-best-test_2d_normalize.parquet


,,,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
fold,k,window_size,,,,,,,,,,,,,,,,,,,,,
2022-02-02,1,7d,2022-02-02,0.000487,0.002602,3,2,2.000000,0.500000,0.333333,0.200000,0.100000,...,1.0,1.0,1.0,1.000000,1.00,1.00,1.00,1.00,1.00,0.134514
2022-02-04,1,14d,2022-02-04,0.000389,0.005208,2,1,1.500000,0.500000,0.333333,0.200000,0.100000,...,1.0,1.0,1.0,1.000000,1.00,1.00,1.00,1.00,1.00,0.134949
2022-02-06,1,7d,2022-02-06,0.000305,0.003576,2,1,1.750000,1.000000,0.500000,0.300000,0.150000,...,1.0,1.0,1.0,0.750000,1.00,1.00,1.00,1.00,1.00,0.132032
2022-02-08,2,30d,2022-02-08,0.000354,0.007169,3,1,2.333333,0.666667,0.444444,0.266667,0.133333,...,1.0,1.0,1.0,0.805556,1.00,1.00,1.00,1.00,1.00,0.132216
2022-02-10,1,30d,2022-02-10,0.000446,0.004372,3,1,2.200000,0.400000,0.400000,0.240000,0.120000,...,1.0,1.0,1.0,0.750000,0.75,0.75,0.75,0.75,0.75,0.131200
2022-02-12,2,10YE,2022-02-12,0.000692,0.004177,2,1,1.500000,0.750000,0.333333,0.200000,0.100000,...,1.0,1.0,1.0,1.000000,1.00,1.00,1.00,1.00,1.00,0.131574
2022-02-14,2,10YE,2022-02-14,0.000658,0.003441,1,1,1.000000,1.000000,0.333333,0.200000,0.100000,...,1.0,1.0,1.0,1.000000,1.00,1.00,1.00,1.00,1.00,0.128079
2022-02-16,5,90d,2022-02-16,0.000501,0.005204,2,1,1.833333,0.333333,0.333333,0.200000,0.100000,...,1.0,1.0,1.0,1.000000,1.00,1.00,1.00,1.00,1.00,0.129770
2022-02-18,1,21d,2022-02-18,0.000441,0.002474,2,1,1.500000,1.000000,0.333333,0.200000,0.100000,...,1.0,1.0,1.0,1.000000,1.00,1.00,1.00,1.00,1.00,0.128690


In [22]:
next_hparams.describe()

,fold_t,time_train,time_rec,open_proposals,min_recs,avg_recs,precision@1,precision@3,precision@5,precision@10,...,recall@10,recall@15,recall@100,r-precision@1,r-precision@3,r-precision@5,r-precision@10,r-precision@15,r-precision@100,time_eval
count,9,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.0,9.0,9.0,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,2022-02-10 00:00:00,0.000475,0.004247,2.222222,1.111111,1.735185,0.683333,0.371605,0.222963,0.111481,...,1.0,1.0,1.0,0.922840,0.972222,0.972222,0.972222,0.972222,0.972222,0.131447
min,2022-02-02 00:00:00,0.000305,0.002474,1.000000,1.000000,1.000000,0.333333,0.333333,0.200000,0.100000,...,1.0,1.0,1.0,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.128079
25%,2022-02-06 00:00:00,0.000389,0.003441,2.000000,1.000000,1.500000,0.500000,0.333333,0.200000,0.100000,...,1.0,1.0,1.0,0.805556,1.000000,1.000000,1.000000,1.000000,1.000000,0.129770
50%,2022-02-10 00:00:00,0.000446,0.004177,2.000000,1.000000,1.750000,0.666667,0.333333,0.200000,0.100000,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.131574
75%,2022-02-14 00:00:00,0.000501,0.005204,3.000000,1.000000,2.000000,1.000000,0.400000,0.240000,0.120000,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.132216
max,2022-02-18 00:00:00,0.000692,0.007169,3.000000,2.000000,2.333333,1.000000,0.500000,0.300000,0.150000,...,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.134949
std,NaN,0.000129,0.001473,0.666667,0.333333,0.412348,0.268225,0.062635,0.037581,0.018791,...,0.0,0.0,0.0,0.116847,0.083333,0.083333,0.083333,0.083333,0.083333,0.002357
